In [1]:
from hello.hello3 import HelloApp, NotLoggedInError
import datetime
from hello import hello_logger
import os
import requests
import time

In [2]:
# General functions

def call(f, *a, _quiet=False, **k):
    if not _quiet:
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("Calling %s%s" % (f.__name__, s))
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.ConnectTimeout as e:
            print(str(e))
            print("Make sure you're on the right network!")
        except (BadError, IOError) as e:
            print(str(e))
        else:
            return rv        
            
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    try:
        while time.time() < end:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    try:
        while time.time() < tend:
            time.sleep(5)
    except KeyboardInterrupt:
        pass
    
def wait2(s):
    pass
wait = wait1

In [7]:
# Specific functions

# globals
do_o2_req  = 0
do_n2_req  = 0
ph_co2_req = 0
batch_tmpl = "domcr t %d"
logger = None

def setconfig(g, s, v):
    call(h.setconfig, g, s, v)

def getmv():
    return call(h.gpmv, _quiet=True)
    
def setup_gases():
    global do_o2_req, do_n2_req, ph_co2_req
    setconfig("pH", 'CO2_Manual_Max_(%)', 100)
    setconfig("DO", 'O2_Manual_Max_(%)', 100)
    setconfig("DO", 'N2_Manual_Max_(%)', 100)
    mv = getmv()
    do_o2_req = mv['do']['outputUp']
    do_n2_req = mv['do']['outputDown']
    ph_co2_req = mv['ph']['outputDown']
    
def setup_controls(rpm=18, temp=37):
    setag(rpm)
    settemp(37)   
    
def maingas(v):
    qset(h.setmg, v)
    
def qset(f, v):
    if v > 0:
        call(f, 1, v)
    else:
        call(f, 2, 0)
        
def main_n2(pc):
    setdo(pc, do_o2_req)
    
def micro_n2(pc):
    global ph_co2_req
    if pc > 0:
        call(h.setph, 1, pc, 0)
    else:
        call(h.setph, 2, ph_co2_req, 0)
    ph_co2_req = pc
    
def phoff():
    call(h.setph, 2, 0, 0)
    
def micro_o2(pc):
    setdo(do_n2_req, pc)
    
def setdo(n2, o2):
    global do_n2_req, do_o2_req
    call(h.setdo, 1, n2, o2)
    do_o2_req = o2
    do_n2_req = n2
    
def setag(rpm):
    call(h.setag, 0, rpm)
    
def settemp(t):
    call(h.settemp, 0, t)
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
    
def startbatch(n):
    endbatch()
    call(h.startbatch, n)
    
def docal1pt():
    v = call(h.getrawvalue, "doa")
    call(h.trycal, 'doa', v, 100)
    call(h.savetrialcal, 'doa')

def dopv():
    return getmv()['do']['pv']

def waitdo_lower(t, timeout=8*3600):
    while dopv() > t and timeout>0:
        wait3(60, 'waitdo_lower')
        timeout -= 60
    print()
    
def dooff():
    call(h.setdo, 2, 0, 0)
    
def waitdo_higher(t, timeout=8*3600):
    while dopv() < t and timeout>0:
        wait3(60, 'waitdo_higher')
        timeout -= 60
    print()
    
def rampdown(mg=0.5, min=20, timeout=2*3600):
    maingas(mg)
    micro_n2(100)
    dooff()
    waitdo_lower(min, timeout)
    phoff()
    
def flushheadspace(gas='air', mg=0.5, wt=5*60):
    maingas(mg)
    if gas == "n2":
        main_n2(100)
    else:
        main_n2(0)
    wait(wt)
    dooff()
    
def rampup(mg=0.5, o2=20, max=80, timeout=2*3600):
    maingas(mg)
    micro_o2(o2)
    micro_n2(0)
    main_n2(0)
    waitdo_higher(max, timeout)
    dooff()
    
def newbatch(i, j):
    startbatch(batch_tmpl%(i,j))

In [8]:
def setup_logger():
    global logger
    keys = [
        'temperature.pv',
        'agitation.pv',
        'ph.outputDown',
        'ph.manDown',
        'do.manUp',
        'do.outputDown',
        'do.pv',
        'do.manDown',
        'do.outputUp',
        'maingas.man',
        'maingas.mode',
        'maingas.pv',
        'MFCs.air',
        'MFCs.n2',
        'MFCs.o2',
        'MFCs.co2',
        'level.pv'
    ]
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, 10, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)

In [9]:
def calibrate(t):
    maingas(0.5)
    micro_o2(100)
    micro_n2(0)
    main_n2(0)
    if t < 3600:
        wait(t/2)
        micro_o2(0)
        wait(t/2)
    else:
        wait(t-3600)
        micro_o2(0)
        wait(3600)
    docal1pt()

### Test #1

In [10]:
connect('192.168.1.81')
setup_gases()
setup_controls()
setup_logger()
calibrate(10*3600)
def bt(): return datetime.datetime.now().strftime("%y%m%d")
fn_tmpl = "DO micro test %s %s.csv"
path = "C:\\Users\\natha\\Documents\\test\\do dip tube many maingas"
os.makedirs(path, exist_ok=True)
os.chdir(path)
def new_file(s, i):
    logfile(fn_tmpl%("%s %s"%(s, i), bt()))

def dotest(mg, o2, i):
    new_file("rampdown", i)
    rampdown(mg)
    endlog()
    flushheadspace()
    new_file("rampup", i)
    rampup(mg, o2)
    endlog()
    dooff()

for mg in (0.1, 0.2, 0.3, 0.4, 0.5):
    for o2 in (20, 40, 60, 80, 100):
        dotest(mg, o2, "mg %.1f o2 %d%%" %(mg, o2))
    

Calling setconfig('pH', 'CO2_Manual_Max_(%)', 100)
Calling setconfig('DO', 'O2_Manual_Max_(%)', 100)
Calling setconfig('DO', 'N2_Manual_Max_(%)', 100)
Calling setag(0, 18)
Calling settemp(0, 37)
Calling setmg(1, 0.5)
Calling setdo(1, 0, 100)
Calling setph(2, 0, 0)
Calling setdo(1, 0, 100)
Sleeping 32400 seconds, ending at 08/04/2017 02:26:22 AM
Calling setdo(1, 0, 0)
Sleeping 3600 seconds, ending at 08/04/2017 03:26:24 AM
Calling getRawValue('doa')
Calling trycal('doa', 101.131241, 100)
Calling savetrialcal('doa')
Creating new file: 'DO micro test rampdown mg 0.1 o2 20% 170804.csv'
Calling setmg(1, 0.1)
Calling setdo(2, 0, 0)
Calling setph(1, 100, 0)
Calling setdo(1, 0, 0)
Calling setdo(1, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/04/2017 04:02:31 AM to 08/04/2017 04:03:31 AM          
Calling setph(2, 0, 0)
Calling setdo(2, 0, 0)
Calling setmg(2, 0)
Calling setmg(1, 0.5)
Calling setdo(1, 0, 0)
Sleeping 300 seconds, ending at 08/04/2017 04:08:32 AM
Calling setmg(2, 0)
Calling se

(waitdo_lower) Sleeping 60 seconds from 08/05/2017 08:03:31 PM to 08/05/2017 08:04:31 PM          
Calling setph(2, 0, 0)
Calling setdo(2, 0, 0)
Calling setmg(2, 0)
Calling setmg(1, 0.5)
Calling setdo(1, 0, 0)
Sleeping 300 seconds, ending at 08/05/2017 08:09:32 PM
Calling setmg(2, 0)
Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampup mg 0.5 o2 20% 170805.csv'
Calling setmg(1, 0.5)
Calling setdo(1, 0, 20)
Calling setph(2, 100, 0)
Calling setdo(1, 0, 20)
(waitdo_higher) Sleeping 60 seconds from 08/05/2017 09:27:43 PM to 08/05/2017 09:28:43 PM          
Calling setdo(2, 0, 0)
Creating new file: 'DO micro test rampdown mg 0.5 o2 40% 170805.csv'
Calling setmg(1, 0.5)
Calling setdo(2, 0, 0)
Calling setph(1, 100, 0)
Calling setdo(1, 0, 0)
Calling setdo(1, 0, 0)
(waitdo_lower) Sleeping 60 seconds from 08/05/2017 10:05:51 PM to 08/05/2017 10:06:51 PM          
Calling setph(2, 0, 0)
Calling setdo(2, 0, 0)
Calling setmg(2, 0)
Calling setmg(1, 0.5)
Calling setdo(1, 0, 0)
Sleeping 300

In [28]:
_10

NameError: name '_10' is not defined